# 공고문 요약 및 Q&A 봇

In [8]:
!pip install langchain langchain-openai langchain-pinecone langchain-community pypdf langchain-chroma

In [1]:
from google.colab import userdata
import os

os.environ['LANGSMITH_TRACING'] = userdata.get('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [2]:
os.environ['OPENAI_EMBEDDING_MODEL'] = userdata.get('OPENAI_EMBEDDING_MODEL')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

## PDF 로드 및 텍스트 추출
다량의 그림과 도표가 있는 공고문, 38장까지

In [15]:
from langchain_community.document_loaders import PyPDFLoader

def load_pdf(path: str):
  loader = PyPDFLoader('./notice.pdf')
  docs = loader.load()
  return docs

docs = load_pdf('./notice.pdf')
for doc in docs[:3]:
  print(doc.metadata)
  print(doc.page_content)
  print()

{'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 16.0 (Macintosh)', 'creationdate': '2020-12-18T09:50:46+09:00', 'moddate': '2020-12-18T11:26:06+09:00', 'trapped': '/False', 'source': './notice.pdf', 'total_pages': 38, 'page': 0, 'page_label': '1'}
참여자 안내서
구직자를 위한 고용안전망
구직자를 위한 고용안전망

{'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 16.0 (Macintosh)', 'creationdate': '2020-12-18T09:50:46+09:00', 'moddate': '2020-12-18T11:26:06+09:00', 'trapped': '/False', 'source': './notice.pdf', 'total_pages': 38, 'page': 1, 'page_label': '2'}
목차
국민취업지원제도 참여자 안내서
02
국민취업지원제도 주요 용어 설명 03
Ⅰ. 한눈에 살펴보는 국민취업지원제도 05
  1. 국민취업지원제도  06
  2. 지원대상 07
  3. 지원내용 10 
  4. 신청 및 접수 방법 11
  5. 지원 절차 12
  6. 참여자의 기본의무와 의무위반 결과 13
  7. 신청에 대한 통지, 결정에 대한 이의제기 14
Ⅱ. 참여자가 제공받는 취업지원서비스 및 생계안정 지원  15
  1. 취업지원서비스 제공 16
  2. 생계안정을 위한 소득 지원 17
  3. 사후관리 지원 19
Q&A 20
유의사항(개인정보보호) 28
취업지원 프로그램  29

{'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 16.0 (Macintosh)', 'creationda

## Document Transformer: 전처리 & 분할(chunking)

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

# 국민 취업지원 제도 경우
def clean_text(text: str) -> str:
  """헤더/페이지번호 삭제"""
  text = re.sub(r"국민취업지원제도 참여자 안내서", " ", text)
  text = re.sub(r"\s+", " ", text).strip()
  return text

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=['\n\n', '\n', '.', ' ', '']
)

clean_docs = [
    doc.copy(update={'page_content': clean_text(doc.page_content)})
    for doc in docs
]

chunks = splitter.split_documents(clean_docs)
print(f'총 {len(chunks)}개 chunk 생성')

총 70개 chunk 생성


/tmp/ipython-input-21-490485387.py:18: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  doc.copy(update={'page_content': clean_text(doc.page_content)})


In [22]:
print(len(docs))

for i, doc in enumerate(docs):
  print(f'{i}: page {doc.metadata["page"]}')
  print(doc.page_content)

38
0: page 0
참여자 안내서
구직자를 위한 고용안전망
구직자를 위한 고용안전망
1: page 1
목차
국민취업지원제도 참여자 안내서
02
국민취업지원제도 주요 용어 설명 03
Ⅰ. 한눈에 살펴보는 국민취업지원제도 05
  1. 국민취업지원제도  06
  2. 지원대상 07
  3. 지원내용 10 
  4. 신청 및 접수 방법 11
  5. 지원 절차 12
  6. 참여자의 기본의무와 의무위반 결과 13
  7. 신청에 대한 통지, 결정에 대한 이의제기 14
Ⅱ. 참여자가 제공받는 취업지원서비스 및 생계안정 지원  15
  1. 취업지원서비스 제공 16
  2. 생계안정을 위한 소득 지원 17
  3. 사후관리 지원 19
Q&A 20
유의사항(개인정보보호) 28
취업지원 프로그램  29
2: page 2
03
국민취업지원제도 주요 용어 설명
  (취업지원) 수급자의 취업활동에 도움이 될 수 있는 취업지원서비스를 제공하고 구직촉진수당을  
지급하는 것(법 제2조제1호)
  (취업지원서비스) 수급자의 취업활동에 도움이 될 수 있도록 취업활동계획 수립, 취업지원 및  
구직활동지원 프로그램 등 구직욕구 및 취업역량을 제고시키기 위한 지원을 제공하는 것  
(법 제12조 ~제14조)
  (구직촉진수당) 수급자의 구직활동 및 생활안정에 소요되는 비용을 지원하기 위하여 법 제7조에  
따른 수급 자격을 인정받은 사람이 구직활동의무를 이행한 경우에 지급하는 수당(법 제18조)
  (취업활동비용) 구직촉진수당 수급요건에 해당하지 않는 수급자격자가 취업지원서비스에 참여하기 
위해 필요하다고 인정해 지급하는 비용(법 제16조)
  (취업성공수당) 수급자가 취업지원서비스 기간 및 사후관리 기간 중에 취업하여 이를 유지하는  
경우 지급하는 수당(법 제17조)
  (수급자격자) 취업지원서비스 또는 구직촉진수당의 수급요건을 갖추어 수급자격이 인정된 사람 
(법 제2조제2호)
  (수급자) 수급자격자로서 취업지원서비스와 구직촉진수당을 받는 사람(법 제2조제3호)
  (취업활동계

## 임베딩 및 벡터스토어 저장

In [6]:
# 임베딩 모델
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

In [7]:
# 벡터 스토어 생성
from langchain_chroma import Chroma

vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embeddings
)
vector_store

## 1) 요약 체인 (템플릿화)

In [43]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from IPython.display import Markdown, display

summary_prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 정부 지원 공고문을 핵심만 뽑아 알려주는 정확하고 친절한 챗봇입니다.'),
    ('human', """
다음 공고문 내용을 항목별로 요약해 주세요.
* 제공된 context만을 참고하여 요약해 주세요.
* 공고문을 요약할 때 context에서 확인할 수 없는 내용에 대해서 지어내거나 상상해서 이야기 하지 마세요.
* 모르는 내용에 대해서는 '해당 질문은 공고문에서 확인할 수 없습니다. 문의처로 문의 바랍니다.'라고 답변해 주세요.

공고문:
{context}

[최종 응답형식]
- 공고명:\n\n
- 신청대상:\n\n
- 신청기간:\n\n
- 지원내용:\n\n
- 제출서류:\n\n
- 문의처:\n\n

참조문서:
- <<source>> (page: <<page>>): <<page_content>>
    """)
])

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.2)
# 요약 체인이므로 전체문서 연결
context_text = '\n\n'.join([chunk.page_content for chunk in chunks])

summary_chain = summary_prompt | llm
summary_output = summary_chain.invoke({'context': context_text})
display(Markdown(summary_output.content))

- 공고명: 국민취업지원제도

- 신청대상: 취업을 원하는 청년, 장기실업자, 경력단절여성, 저소득 구직자, 특수형태근로종사자 등 취업취약계층

- 신청기간: 해당 질문은 공고문에서 확인할 수 없습니다. 문의처로 문의 바랍니다.

- 지원내용: 
  1. Ⅰ유형: 구직촉진수당(최대 300만원) 및 취업지원서비스 제공
  2. Ⅱ유형: 취업지원서비스 및 취업활동비용 지원
  3. 취업지원서비스 기간: 최대 1년, 필요시 6개월 연장 가능
  4. 취업성공수당(최대 150만원) 지급

- 제출서류: 
  1. 취업지원 신청서
  2. 개인정보 수집·이용 및 제공 동의서
  3. 필요시 가구단위 증빙서류 및 특정 취약계층 증빙서류

- 문의처: 해당 질문은 공고문에서 확인할 수 없습니다. 문의처로 문의 바랍니다.

## 2) RAG 질의응답 체인

In [41]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

retriever = vector_store.as_retriever(search_kwargs={'k':3})

qa_prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 공고문의 내용만 근거로 답변하는 정확하고 친절한 챗봇입니다.'),
    ('human', """
* 제공된 context만을 참고하여 다음 질문에 답변해주세요.
* 공고문을 요약할 때 context에서 확인할 수 없는 내용에 대해서 지어내거나 상상해서 이야기 하지 마세요.
* 모르는 내용에 대해서는 '해당 질문은 공고문에서 확인할 수 없습니다. 문의처로 문의 바랍니다.'라고 답변해 주세요.
* 최종 응답에는 참조한 context에 대한 정보를 추가해주세요.

[사용자 질문]
{query}

[참고 문서]
{context}

[최종 응답형식]
답변:

참조문서:
- <<source>> (page: <<page>>): <<page_content>>

    """)
])

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.3)
output_parsers = StrOutputParser()
qa_chain = (
    {'context': retriever, 'query': RunnablePassthrough()}
    | qa_prompt | llm | output_parsers
)

In [35]:
user_input = '신청 대상은 누구야?'

if user_input:
  response = qa_chain.invoke(user_input)
  print(response)

답변: 신청 대상은 저소득층 등 청년(18~34세), 중장년(35~69세)으로, 소득 기준은 중위소득 50% 이하, 재산 기준은 3억원 이하입니다. 또한, 특정계층으로는 생계급여 수급자, 실업급여 수급 중이거나 종료 후 6개월이 지나지 않은 자, 자치단체 청년수당 수급 중이거나 종료 후 6개월이 지나지 않은 자 등이 포함됩니다. 

참조문서:
- <<source>> (page: 10): 04 참여를 희망하는 본인이 직접 거주지 관할 고용센터에 방문하거나 온라인(www.work.go.kr/kua)을 통해 신청하실 수 있습니다. “신청 및 접수” 방법은? 11 Ⅰ유형과 Ⅱ유형 비교 구분 Ⅰ유형 Ⅱ유형 요건심사형 선발형 저소득층 등 청년 중장년 청년 비경활 저소득층 특정계층 지원 대상 연령 15~69세 (청년: 18~34세, 중장년: 35~69세) 소득 중위소득 50%↓ 중위소득 120%↓ 중위소득 50%↓ 중위소득 60%↓ × × 중위소득 100%↓ 재산 3억원↓ 3억원↓ *고시로 규정 가능 3억원↓ × 취업경험 2년이내 100일 (800시간) 이상 × × × 지원 내용 취업지원서비스 ○ 소 득 지 원 구직촉진수당 ○ × 취업활동비용 × ○ 신청인은 ❶ 취업지원 신청서와 지원자격 확인을 위한 ❷ 개인정보 수집·이용 및 제공 동의서를 반드시 제출하여야 합니다. 대부분의 지원자격 정보는 개인정보 활용 동의를 통해 공적시스템에서 파악할 수 있습니다. 그 밖에 가구원 확정, 전산망으로 확인이 어렵거나 실시간으로 파악되지 않는 소득·재산 등의 관련 정보는, 신청인 본인이 직접 증빙서류를 제출해 입증하면 됩니다. * (필요시) 가구단위 증빙서류 : 가족관계증명원, 실종신고서 특정 취약계층 증빙서류 : 관련 추천서, 확인서 전산망으로 확인불가 또는 전산망에 실시간 연계되지 않은 소득·재산·취업경험 관련 정보 : 사업주 확인자료 등 관련 증빙자료 Ⅰ. 한눈에 살펴보는 국민취업지원제도
